In [1]:
import pickle
import pandas as pd
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# Obtener la ruta del directorio base (nivel superior del proyecto)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Agregar SRC_PATH a sys.path si no está presente
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

In [3]:
with open(r'..\models\RandomForestClassifier.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)


In [4]:
df_oot = pd.read_csv(r'..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')
df_tr = pd.read_csv(r"..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_trtest.csv")
features = pd.read_csv(r'..\data\procesed\features.csv')

df_oot['id'] = df_oot['nit_enmascarado'].astype(str) + '#' + df_oot['num_oblig_orig_enmascarado'].astype(str) + '#' + df_oot['num_oblig_enmascarado'].astype(str)
df_tr['id'] = df_tr['nit_enmascarado'].astype(str) + '#' + df_tr['num_oblig_orig_enmascarado'].astype(str) + '#' + df_tr['num_oblig_enmascarado'].astype(str)

df_tr = df_tr.drop_duplicates(subset='id', keep='first')
df = pd.merge(df_oot, df_tr, on='id', how='left')

In [5]:
from data_engineer import FeatureSelector

limpieza = FeatureSelector(df, features)
df = limpieza.fit_transform()

Limpiando variables numéricas...


Separando X e y...
Detectando tipos de variables...
Preprocesando variables...
Transformando variables...
Reconstruyendo nombres de columnas...
Seleccionando variables...
Ajustando columnas faltantes...
Ordenando columnas...
Finalizando DataFrame...


In [6]:
# 2. Seleccionar columnas que usó el modelo
columnas_modelo = modelo.feature_names_in_
X = df[columnas_modelo]

# 3. Predecir
predicciones = modelo.predict(X)

# 4. Crear DataFrame final
df_submission = pd.DataFrame({
    'id': df['id'],
    'var_rpta_alt': predicciones,
    'var_rpta_alt_prob': modelo.predict_proba(X)[:, 1]
})

# 5. Guardar en CSV con el formato solicitado
df_submission.to_csv('submission.csv', index=False)


In [7]:
df_submission

,id,var_rpta_alt,var_rpta_alt_prob
0,257335#444821#635511,1,0.89
1,59584#350400#730364,1,0.89
2,397604#973821#106521,1,1.00
3,368086#382995#696856,1,1.00
4,255009#434238#645924,1,0.51
...,...,...,...
112544,316499#14503#1074624,1,0.96
112545,254311#30479#1013957,1,0.51
112546,483193#13732#1006779,1,0.89
112547,293824#22004#1062743,1,0.93
